In [ ]:
#| default_exp visualization

In [ ]:
#| hide
from nbdev.showdoc import *

## Overlay over predicted edges on target images

In [ ]:
#| export
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
from skimage.feature import canny
from torchvision.utils import make_grid

In [ ]:
#| exporti
def _overlay_edges(target, pred, sigma, eps=1e-5):
    pred = (pred - pred.min()) / (pred.max() - pred.min() + eps)
    edges = canny(pred, sigma=sigma)
    edges = np.ma.masked_where(~edges, edges)

    buffer = BytesIO()
    plt.subplot()
    plt.imshow(target, cmap="gray")
    plt.imshow(edges, cmap="cool_r", interpolation="none", vmin=0.0, vmax=1.0)
    plt.axis("off")
    plt.savefig(buffer, format="png", bbox_inches="tight", pad_inches=0, dpi=300)
    arr = np.array(Image.open(buffer))
    plt.close()
    return arr

In [ ]:
#| export
def overlay_edges(target, pred, sigma=1.5):
    """Generate edge overlays for a batch of targets and predictions."""
    edges = []
    for i, p in zip(target, pred):
        edge = _overlay_edges(i[0].cpu().numpy(), p[0].cpu().numpy(), sigma)
        edges.append(edge)
    edges = torch.from_numpy(np.stack(edges)).permute(0, -1, 1, 2)
    edges = make_grid(edges).permute(1, 2, 0)
    return edges

## Using PyVista to visualize 3D geometry

In [ ]:
#| export
import pyvista
from torch.nn.functional import pad

from diffpose.calibration import RigidTransform, perspective_projection

In [ ]:
#| exporti
def fiducials_3d_to_projected_fiducials_3d(specimen, pose):
    # Extrinsic camera matrix
    extrinsic = (
        specimen.lps2volume.inverse()
        .compose(pose.inverse())
        .compose(specimen.translate)
        .compose(specimen.flip_xz)
    )

    # Intrinsic projection -> in 3D
    x = perspective_projection(extrinsic, specimen.intrinsic, specimen.fiducials)
    x = -specimen.focal_len * torch.einsum(
        "ij, bnj -> bni",
        specimen.intrinsic.inverse(),
        pad(x, (0, 1), value=1),  # Convert to homogenous coordinates
    )

    # Some command-z
    extrinsic = (
        specimen.flip_xz.inverse().compose(specimen.translate.inverse()).compose(pose)
    )
    return extrinsic.transform_points(x)

In [ ]:
#| export
def fiducials_to_mesh(
    specimen,
    rotation=None,
    translation=None,
    parameterization=None,
    convention=None,
    detector=None,
):
    """
    Use camera matrices to get 2D projections of 3D fiducials for a given pose.
    If the detector is passed, 2D projections will be filtered for those that lie
    on the detector plane.
    """
    # Location of fiducials in 3D
    fiducials_3d = specimen.lps2volume.inverse().transform_points(specimen.fiducials)
    fiducials_3d = pyvista.PolyData(fiducials_3d.squeeze().numpy())
    if rotation is None and translation is None and parameterization is None:
        return fiducials_3d

    # Embedding of fiducials in 2D
    pose = RigidTransform(rotation, translation, parameterization, convention, device="cpu")
    fiducials_2d = fiducials_3d_to_projected_fiducials_3d(specimen, pose)
    fiducials_2d = fiducials_2d.squeeze().numpy()

    # Optionally, only render 2D fiducials that lie on the detector plane
    if detector is not None:
        corners = detector.points.reshape(
            detector["height"][0], detector["width"][0], 3
        )[
            [0, 0, -1, -1],
            [0, -1, 0, -1],
        ]
        exclude = np.logical_or(
            fiducials_2d < corners.min(0),
            fiducials_2d > corners.max(0),
        ).any(1)
        fiducials_2d = fiducials_2d[~exclude]

    fiducials_2d = pyvista.PolyData(fiducials_2d)
    return fiducials_3d, fiducials_2d

In [ ]:
#| export
def lines_to_mesh(camera, fiducials_2d):
    """Draw lines from the camera to the 2D fiducials."""
    lines = []
    for pt in fiducials_2d.points:
        line = pyvista.Line(pt, camera.center)
        lines.append(line)
    return lines

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()